# Purpose
The purpose of this notebook is to compare the case study predictive results for the MNL model with systematic heterogeneity to the log-normal MIXL model from Brownstone and Train (1998).

In [1]:
import sys
from copy import deepcopy
from collections import OrderedDict
from collections import defaultdict

import scipy.stats
import pandas as pd
import numpy as np

import pylogit as pl

sys.path.insert(0, '../src/')
from visualization import predictive_viz as viz

%matplotlib inline

/Users/timothyb0912/anaconda/lib/python2.7/site-packages/IPython/html.py:14: ShimWarning: The `IPython.html` package has been deprecated. You should import from `notebook` instead. `IPython.html.widgets` has moved to `ipywidgets`.
  "`IPython.html.widgets` has moved to `ipywidgets`.", ShimWarning)


# Load the car data

In [2]:
car_df = pd.read_csv("../data/processed/model_ready_car_data.csv")
forecast_df = pd.read_csv("../data/processed/forecast_car_data.csv")

# Create the model specification

In [3]:
def create_specification_dict(list_of_cols_and_names):
    # Create interaction variables for the various body types
    body_names = ['sports_utility_vehicle',
                  'sports_car',
                  'station_wagon',
                  'truck',
                  'van']

    non_body_or_fuel_vars = ['price_over_log_income',
                             'price_over_log_income_lte_3',
                             'price_over_log_income_gt_3',
                             'range_over_100',
                             'acceleration_over_10',
                             'top_speed_over_100',
                             'pollution',
                             'vehicle_size_over_10',
                             'tens_of_cents_per_mile']

    body_interactions = defaultdict(lambda : [])

    for body in body_names:
        for interaction_var in non_body_or_fuel_vars:
            new_name = interaction_var + "_for_" + body
            # Store the new variable name
            body_interactions[interaction_var].append(new_name)

    # Create interaction variables for the various fuel types
    fuel_names = ['electric',
                  'compressed_natural_gas',
                  'methanol']

    fuel_interaction_vars = ['price_over_log_income',
                             'price_over_log_income_lte_3',
                             'price_over_log_income_gt_3',
                             'range_over_100',
                             'top_speed_over_100',
                             'pollution',
                             'vehicle_size_over_10',
                             'tens_of_cents_per_mile']

    fuel_interactions = defaultdict(lambda : [])

    for fuel in fuel_names:
        for interaction_var in fuel_interaction_vars:
            new_name = interaction_var + "_for_" + fuel
            # Store the new variable name
            fuel_interactions[interaction_var].append(new_name)
            
    # Create specification and name objects
    spec_dict, name_dict = OrderedDict(), OrderedDict()
            
    for col, display_name in list_of_cols_and_names:
        if col in body_interactions:
            for interaction_col in body_interactions[col]:
                suffix = interaction_col[interaction_col.rfind("for_") + 4:]
                new_display_name = display_name + " ({})".format(suffix)

                if car_df[interaction_col].unique().size == 1:
                    continue

                spec_dict[interaction_col] = 'all_same'
                name_dict[interaction_col] = new_display_name

            for interaction_col in fuel_interactions[col]:
                suffix = interaction_col[interaction_col.rfind("for_") + 4:]
                new_display_name = display_name + "({})".format(suffix)

                if car_df[interaction_col].unique().size == 1:
                    continue

                spec_dict[interaction_col] = 'all_same'
                name_dict[interaction_col] = new_display_name

        spec_dict[col] = 'all_same'
        name_dict[col] = display_name
        
    return spec_dict, name_dict


In [4]:
orig_cols_and_display_names =\
    [('price_over_log_income_lte_3', 'Price over log(income) <= 3'),
     ('price_over_log_income_gt_3', 'Price over log(income) > 3'),
     ('range_over_100', 'Range (units: 100mi)'),
     ('acceleration_over_10', 'Acceleration (units: 0.1sec)'),
     ('top_speed_over_100', 'Top speed (units: 0.01mph)'),
     ('pollution', 'Pollution'),
     ('vehicle_size_over_10', 'Size'),
     ('big_enough', 'Big enough'),
     ('luggage_space', 'Luggage space'),
     ('tens_of_cents_per_mile', 'Operation cost'),
     ('station_availability', 'Station availability'),
     ('sports_utility_vehicle', 'Sports utility vehicle'),
     ('sports_car', 'Sports car'),
     ('station_wagon', 'Station wagon'),
     ('truck', 'Truck'),
     ('van', 'Van'),
     ('electric', 'EV'),
     ('electric_commute_lte_5mi', 'Commute < 5 & EV'),
     ('electric_and_college', 'College & EV'),
     ('compressed_natural_gas', 'CNG'),
     ('methanol', 'Methanol'),
     ('methanol_and_college', 'College & Methanol')]
    
interaction_mnl_spec, interaction_mnl_names =\
    create_specification_dict(orig_cols_and_display_names)

# Estimate the expanded and original MNL models

In [6]:
# Determine the number of index coefficients for the interaction MNL
num_index_coefs = len(interaction_mnl_names)

# Initialize the interaction mnl model object
interaction_model =\
    pl.create_choice_model(data=car_df,
                           alt_id_col='alt_id',
                           obs_id_col='obs_id',
                           choice_col='choice',
                           specification=interaction_mnl_spec,
                           model_type='MNL',
                           names=interaction_mnl_names)
    
interaction_model.fit_mle(np.zeros(num_index_coefs))

interaction_model.get_statsmodels_summary()

Log-likelihood at zero: -8,338.8486
Initial Log-likelihood: -8,338.8486


/Users/timothyb0912/anaconda/lib/python2.7/site-packages/scipy/optimize/_minimize.py:507: RuntimeWarning: Method BFGS does not use Hessian information (hess).
  RuntimeWarning)


Estimation Time for Point Estimation: 2.14 seconds.
Final log-likelihood: -7,311.6340


<class 'statsmodels.iolib.summary.Summary'>
"""
                     Multinomial Logit Model Regression Results                    
===================================================================================
Dep. Variable:                      choice   No. Observations:                4,654
Model:             Multinomial Logit Model   Df Residuals:                    4,572
Method:                                MLE   Df Model:                           82
Date:                     Sun, 10 Jun 2018   Pseudo R-squ.:                   0.123
Time:                             17:21:06   Pseudo R-bar-squ.:               0.113
AIC:                            14,787.268   Log-Likelihood:             -7,311.634
BIC:                            15,315.798   LL-Null:                    -8,338.849
=========================================================================================================================
                                                            coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------------------------------------------------
Price over log(income) <= 3 (sports_utility_vehicle)      0.2328      0.257      0.905      0.366      -0.271       0.737
Price over log(income) <= 3 (sports_car)                 -0.1069      0.253     -0.423      0.673      -0.603       0.389
Price over log(income) <= 3 (station_wagon)              -0.3201      0.109     -2.937      0.003      -0.534      -0.106
Price over log(income) <= 3 (truck)                      -0.2541      0.089     -2.863      0.004      -0.428      -0.080
Price over log(income) <= 3 (van)                        -0.2418      0.088     -2.741      0.006      -0.415      -0.069
Price over log(income) <= 3(electric)                    -0.1319      0.100     -1.324      0.186      -0.327       0.063
Price over log(income) <= 3(compressed_natural_gas)      -0.0938      0.092     -1.024      0.306      -0.273       0.086
Price over log(income) <= 3(methanol)                     0.0629      0.089      0.707      0.480      -0.112       0.237
Price over log(income) <= 3                              -0.2519      0.113     -2.225      0.026      -0.474      -0.030
Price over log(income) > 3 (sports_utility_vehicle)       0.2714      0.077      3.508      0.000       0.120       0.423
Price over log(income) > 3 (sports_car)                   0.2039      0.080      2.534      0.011       0.046       0.362
Price over log(income) > 3 (station_wagon)               -0.0341      0.047     -0.722      0.470      -0.127       0.059
Price over log(income) > 3 (truck)                       -0.0087      0.035     -0.249      0.804      -0.077       0.060
Price over log(income) > 3 (van)                         -0.0400      0.036     -1.110      0.267      -0.110       0.031
Price over log(income) > 3(electric)                     -0.0941      0.035     -2.655      0.008      -0.164      -0.025
Price over log(income) > 3(compressed_natural_gas)       -0.0534      0.033     -1.616      0.106      -0.118       0.011
Price over log(income) > 3(methanol)                     -0.0699      0.031     -2.253      0.024      -0.131      -0.009
Price over log(income) > 3                               -0.1326      0.037     -3.553      0.000      -0.206      -0.059
Range (units: 100mi) (sports_utility_vehicle)            -0.0964      0.100     -0.965      0.334      -0.292       0.099
Range (units: 100mi) (sports_car)                        -0.0721      0.117     -0.616      0.538      -0.302       0.157
Range (units: 100mi) (station_wagon)                     -0.0725      0.086     -0.848      0.397      -0.240       0.095
Range (units: 100mi) (truck)                              0.0125      0.070      0.178      0.859      -0.125       0.150
Range (units: 100mi) (van)                                0.0628      0.067      0.939      0.348      -0.068       0.194
Range (units: 100mi)(ele

In [7]:
car_mnl_spec, car_mnl_names = OrderedDict(), OrderedDict()

cols_and_display_names =\
    [('price_over_log_income', 'Price over log(income)'),
     ('range_over_100', 'Range (units: 100mi)'),
     ('acceleration_over_10', 'Acceleration (units: 0.1sec)'),
     ('top_speed_over_100', 'Top speed (units: 0.01mph)'),
     ('pollution', 'Pollution'),
     ('vehicle_size_over_10', 'Size'),
     ('big_enough', 'Big enough'),
     ('luggage_space', 'Luggage space'),
     ('tens_of_cents_per_mile', 'Operation cost'),
     ('station_availability', 'Station availability'),
     ('sports_utility_vehicle', 'Sports utility vehicle'),
     ('sports_car', 'Sports car'),
     ('station_wagon', 'Station wagon'),
     ('truck', 'Truck'),
     ('van', 'Van'),
     ('electric', 'EV'),
     ('electric_commute_lte_5mi', 'Commute < 5 & EV'),
     ('electric_and_college', 'College & EV'),
     ('compressed_natural_gas', 'CNG'),
     ('methanol', 'Methanol'),
     ('methanol_and_college', 'College & Methanol')]
    
for col, display_name in cols_and_display_names:
    car_mnl_spec[col] = 'all_same'
    car_mnl_names[col] = display_name

# Initialize the mnl model
simple_mnl = pl.create_choice_model(data=car_df,
                                 alt_id_col='alt_id',
                                 obs_id_col='obs_id',
                                 choice_col='choice',
                                 specification=car_mnl_spec,
                                 model_type='MNL',
                                 names=car_mnl_names)

# Create the initial variables for model estimation
num_vars = len(car_mnl_names)
initial_vals = np.zeros(num_vars)

# Estimate the mnl model
fit_vals = simple_mnl.fit_mle(initial_vals,
                              method='L-BFGS-B',
                              just_point=True)['x']
# Note ridge=1e-7 produces the same results as non-regularized MLE
simple_mnl.fit_mle(fit_vals, method='BFGS')

# Look at the estimation results
simple_mnl.get_statsmodels_summary()

/Users/timothyb0912/anaconda/lib/python2.7/site-packages/scipy/optimize/_minimize.py:507: RuntimeWarning: Method L-BFGS-B does not use Hessian information (hess).
  RuntimeWarning)


Log-likelihood at zero: -8,338.8486
Initial Log-likelihood: -7,391.8638
Estimation Time for Point Estimation: 0.14 seconds.
Final log-likelihood: -7,391.8300


<class 'statsmodels.iolib.summary.Summary'>
"""
                     Multinomial Logit Model Regression Results                    
===================================================================================
Dep. Variable:                      choice   No. Observations:                4,654
Model:             Multinomial Logit Model   Df Residuals:                    4,633
Method:                                MLE   Df Model:                           21
Date:                     Sun, 10 Jun 2018   Pseudo R-squ.:                   0.114
Time:                             17:21:17   Pseudo R-bar-squ.:               0.111
AIC:                            14,825.660   Log-Likelihood:             -7,391.830
BIC:                            14,961.015   LL-Null:                    -8,338.849
================================================================================================
                                   coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------------------------
Price over log(income)          -0.1854      0.027     -6.796      0.000      -0.239      -0.132
Range (units: 100mi)             0.3501      0.027     13.052      0.000       0.298       0.403
Acceleration (units: 0.1sec)    -0.7160      0.111     -6.472      0.000      -0.933      -0.499
Top speed (units: 0.01mph)       0.2612      0.081      3.228      0.001       0.103       0.420
Pollution                       -0.4441      0.102     -4.367      0.000      -0.643      -0.245
Size                             0.9345      0.316      2.953      0.003       0.314       1.555
Big enough                       0.1432      0.077      1.853      0.064      -0.008       0.295
Luggage space                    0.5009      0.191      2.623      0.009       0.127       0.875
Operation cost                  -0.7679      0.076    -10.131      0.000      -0.916      -0.619
Station availability             0.4133      0.096      4.294      0.000       0.225       0.602
Sports utility vehicle           0.8201      0.141      5.830      0.000       0.544       1.096
Sports car                       0.6370      0.148      4.298      0.000       0.347       0.928
Station wagon                   -1.4367      0.062    -23.139      0.000      -1.558      -1.315
Truck                           -1.0168      0.049    -20.753      0.000      -1.113      -0.921
Van                             -0.7989      0.047    -16.864      0.000      -0.892      -0.706
EV                              -0.1786      0.172     -1.041      0.298      -0.515       0.158
Commute < 5 & EV                 0.1983      0.084      2.374      0.018       0.035       0.362
College & EV                     0.4426      0.109      4.058      0.000       0.229       0.656
CNG                              0.3450      0.092      3.744      0.000       0.164       0.526
Methanol                         0.3134      0.103      3.051      0.002       0.112       0.515
College & Methanol               0.2284      0.089      2.576      0.010       0.055       0.402
================================================================================================
"""

# Make predictions

In [8]:
# Create a set of values to use for grouping
grouping_series = [forecast_df.vehicle_size,
                   forecast_df.fuel_type,
                   forecast_df.body_type]

In [9]:
# Get forecast probabilities using the interaction MNL
mnl_forecast_probs =\
    pd.Series(interaction_model.predict(forecast_df))
    
# Get forecast probabilities using the log-normal MIXL model
mixl_forecast_probs =\
    (pd.read_csv("../data/processed/lognormal_mixl_probs_mle_forecast.csv",
                 header=None)
       .iloc[:, 0])
    
# Get the original probabilities using the interaction MNL
mnl_original_probs =\
    pd.Series(interaction_model.long_fitted_probs)

# Get the original probabilities using the log-normal MIXL
mixl_original_probs =\
    pd.read_csv("../data/processed/lognormal_mixl_probs_mle.csv",
                header=None).iloc[:, 0]
    
# Get forecast probabilities using the simple MNL
simple_mnl_forecast_probs =\
    pd.Series(simple_mnl.predict(forecast_df))

# Get the original probabilities using the simple MNL
simple_mnl_original_probs =\
    pd.Series(simple_mnl.long_fitted_probs)

In [10]:
# Ensure the forecast probabilities for large gas cars are
# higher than the original probabilities for large gas cars
large_gas_car_idx = ((car_df['body_type'] == 'regcar') &
                     (car_df['vehicle_size'] == 3) &
                     (car_df['fuel_type'] == 'gasoline')).values
num_stupid_forecasts =\
    ((mixl_forecast_probs >
      mixl_original_probs)[large_gas_car_idx]).sum()
print("{:,} stupid forecasts".format(num_stupid_forecasts))

0 stupid forecasts


In [11]:
# Look at the total number of forecasted observations
# choosing large gas cars under the baseline and increased
# price scenarios with Brownstone and Train's Mixed Logit B
(mixl_original_probs[large_gas_car_idx].sum(),
 mixl_forecast_probs[large_gas_car_idx].sum())

(419.9148930608775, 368.6637927689417)

In [12]:
# Look at the total number of forecasted observations
# choosing large gas cars under the baseline and increased
# price scenarios with the new expanded MNL model.
(mnl_original_probs[large_gas_car_idx].sum(),
 mnl_forecast_probs[large_gas_car_idx].sum())

(420.29898342664444, 386.50889449762315)

In [13]:
# Create a function that will calculate the desired percent
# changes in the predicted mode share
def calc_mode_share_change(orig_prob_series,
                           new_prob_series,
                           grouping_series,
                           num_obs,
                           name=None):
    """
    Calculate the relative change in predicted shares by group.
    """
    new_shares =\
        (new_prob_series.groupby(grouping_series)
                        .agg(np.sum) / num_obs)

    orig_shares =\
        (orig_prob_series.groupby(grouping_series)
                        .agg(np.sum) / num_obs)

    change_in_shares = new_shares - orig_shares
    
    relative_change = change_in_shares / orig_shares

    if isinstance(name, str):
        relative_change.name = name
    
    return relative_change


In [14]:
# Calculate the relative change using the interaction MNL and the
# log-normal MIXL model.
num_obs = interaction_model.nobs
relative_change_mnl =\
    calc_mode_share_change(mnl_original_probs,
                           mnl_forecast_probs,
                           grouping_series,
                           num_obs,
                           name='interaction_mnl')

relative_change_mixl =\
    calc_mode_share_change(mixl_original_probs,
                           mixl_forecast_probs,
                           grouping_series,
                           num_obs,
                           name='lognormal-mixl')

relative_change_simple_mnl =\
    calc_mode_share_change(simple_mnl_original_probs,
                           simple_mnl_forecast_probs,
                           grouping_series,
                           num_obs,
                           name='simple_mnl')

In [15]:
big_change =\
    (((relative_change_mnl >= 2 * relative_change_mixl) & (relative_change_mixl > 0)) |
     ((relative_change_mnl <= 0.5 * relative_change_mixl) & (relative_change_mixl > 0)) |
     ((relative_change_mnl <= 2 * relative_change_mixl) & (relative_change_mixl < 0)) |
     ((relative_change_mnl >= 0.5 * relative_change_mixl) & (relative_change_mixl < 0)))
    
differences =\
    pd.concat([relative_change_mnl.loc[big_change],
               relative_change_mixl.loc[big_change],
               relative_change_simple_mnl.loc[big_change]],
              axis=1)

In [16]:
differences

interaction_mnl  lognormal-mixl  simple_mnl
vehicle_size fuel_type body_type                                             
0            cng       sportcar          0.001834        0.000875    0.001220
3            gasoline  sportcar          0.003110        0.010486    0.004285
                       sportuv           0.004036        0.012243    0.005932
                       stwagon           0.024240        0.061481    0.029491
                       truck             0.028849        0.081567    0.035314
                       van               0.022993        0.057159    0.029298

In [17]:
relative_change_mixl.get_value((3, 'gasoline', 'regcar'))

/Users/timothyb0912/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:1: FutureWarning: get_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  if __name__ == '__main__':


-0.122051161173046

In [18]:
sep = "="
print("Log-normal Mixed Logit")
for size in range(4):
    print("Size {}".format(size))
    print(relative_change_mixl[size])
    print(sep*20)


Log-normal Mixed Logit
Size 0
fuel_type  body_type
cng        regcar       0.000000
           sportcar     0.000875
           sportuv      0.002107
           stwagon      0.000000
           truck        0.000000
           van          0.000000
electric   regcar       0.000000
           sportcar     0.001394
           sportuv      0.000000
           stwagon      0.000000
           truck        0.001105
           van          0.000000
gasoline   regcar       0.000000
           sportcar     0.000000
           sportuv      0.000000
           stwagon      0.000000
           truck        0.000000
           van          0.000000
methanol   regcar       0.000000
           sportcar     0.000000
           sportuv      0.000000
           stwagon      0.000000
           truck        0.000000
           van          0.000000
Name: lognormal-mixl, dtype: float64
Size 1
fuel_type  body_type
cng        regcar       0.003859
           sportcar     0.000843
           sportuv      0.

In [19]:
sep = "="
print("Expanded MNL")
for size in range(4):
    print("Size {}".format(size))
    print(relative_change_mnl[size])
    print(sep*20)


Expanded MNL
Size 0
fuel_type  body_type
cng        regcar       0.000000
           sportcar     0.001834
           sportuv      0.003231
           stwagon      0.000000
           truck        0.000000
           van          0.000000
electric   regcar       0.000000
           sportcar     0.000960
           sportuv      0.000000
           stwagon      0.000000
           truck        0.000683
           van          0.000000
gasoline   regcar       0.000000
           sportcar     0.000000
           sportuv      0.000000
           stwagon      0.000000
           truck        0.000000
           van          0.000000
methanol   regcar       0.000000
           sportcar     0.000000
           sportuv      0.000000
           stwagon      0.000000
           truck        0.000000
           van          0.000000
Name: interaction_mnl, dtype: float64
Size 1
fuel_type  body_type
cng        regcar       0.003662
           sportcar     0.000746
           sportuv      0.001287
  

In [20]:
relative_change_mixl.sort_values(ascending=False).iloc[:10]

vehicle_size  fuel_type  body_type
3             gasoline   truck        0.081567
                         stwagon      0.061481
                         van          0.057159
2             cng        stwagon      0.041434
1             electric   van          0.028276
2             cng        regcar       0.021974
                         truck        0.021644
              methanol   regcar       0.020775
                         van          0.020072
              electric   stwagon      0.018228
Name: lognormal-mixl, dtype: float64

In [21]:
relative_change_mnl.sort_values(ascending=False).iloc[:10]

vehicle_size  fuel_type  body_type
1             electric   van          0.030330
2             cng        stwagon      0.030163
3             gasoline   truck        0.028849
                         stwagon      0.024240
                         van          0.022993
2             electric   stwagon      0.020837
                         van          0.018026
              cng        regcar       0.017354
                         truck        0.017223
1             electric   regcar       0.017089
Name: interaction_mnl, dtype: float64

In [22]:
relative_change_simple_mnl.sort_values(ascending=False).iloc[:10]

vehicle_size  fuel_type  body_type
2             cng        stwagon      0.040856
1             electric   van          0.040792
3             gasoline   truck        0.035314
                         stwagon      0.029491
                         van          0.029298
2             electric   stwagon      0.028574
                         van          0.024009
              cng        truck        0.022911
1             electric   stwagon      0.022452
2             cng        regcar       0.021610
Name: simple_mnl, dtype: float64